Declare of the library usage

In [1]:
import os
from PIL import Image, ImageEnhance
import tensorflow as tf
import numpy as np

2025-01-06 21:56:04.351016: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-06 21:56:04.401405: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-06 21:56:04.648669: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-06 21:56:04.648756: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-06 21:56:04.649784: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
# Check if TensorFlow is built with GPU support
print("TensorFlow GPU support enabled:", tf.test.is_built_with_cuda())

# Check if a GPU device is available
print("GPU device available:", tf.config.list_physical_devices('GPU'))

# Print the list of available GPUs
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        print("GPU Device:", device)
else:
    print("No GPU devices available.")

# Get details of the CUDA and cuDNN versions (if available)
cuda_version = tf.sysconfig.get_build_info()["cuda_version"]
cudnn_version = tf.sysconfig.get_build_info()["cudnn_version"]
print("CUDA version:", cuda_version)
print("cuDNN version:", cudnn_version)

TensorFlow GPU support enabled: True
GPU device available: []
No GPU devices available.
CUDA version: 11.8
cuDNN version: 8


2025-01-06 21:56:25.306855: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-01-06 21:56:25.492836: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
# Paths to your dataset directories
train_dir = "/home/tairo/UVASTRAL/dev/DataSet/Train"
validation_dir = "/home/tairo/UVASTRAL/dev/DataSet/Validation"
test_dir = "/home/tairo/UVASTRAL/dev/DataSet/Test"

In [4]:
# Function to count images in a folder
#Named: count_images
def count_images(directory): #checking for the path to the main directory 
    total_images = 0 #initialize a variables
    for class_folder in os.listdir(directory): 
        class_path = os.path.join(directory, class_folder)
        if os.path.isdir(class_path):
            total_images += len([img for img in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, img))])
    return total_images

In [5]:
# Count and display images before preprocessing
print("Counting images before preprocessing...")
total_train_before = count_images(train_dir)
total_validation_before = count_images(validation_dir)
total_test_before = count_images(test_dir)

print(f"Train: {total_train_before} images")
print(f"Validation: {total_validation_before} images")
print(f"Test: {total_test_before} images")


Counting images before preprocessing...
Train: 0 images
Validation: 0 images
Test: 0 images


In [6]:
# Function to resize all images and perform augmentation
def preprocess_and_augment(directory):
    for class_folder in os.listdir(directory):
        class_path = os.path.join(directory, class_folder)
        
        if not os.path.isdir(class_path):
            continue  # Skip non-directory files
        
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            
            if not os.path.isfile(image_path):
                continue
            
            try:
                # Open image and resize to 224x224
                img = Image.open(image_path).convert('RGB')
                img = img.resize((224, 224))  # All images resized to 224x224
                
                # Overwrite the original image with the resized version
                img.save(image_path)
                
                # Get base name and extension
                base_name, ext = os.path.splitext(image_name)
                
                # Flipping horizontally
                flipped_img = img.transpose(Image.FLIP_LEFT_RIGHT)
                flipped_name = f"{base_name}_flipped{ext}"
                flipped_path = os.path.join(class_path, flipped_name)
                flipped_img.save(flipped_path)
                
                # Rotating (30 degrees clockwise)
                rotated_img = img.rotate(30, expand=True)
                rotated_name = f"{base_name}_rotated{ext}"
                rotated_path = os.path.join(class_path, rotated_name)
                rotated_img.save(rotated_path)
                
                # Brightness enhancement
                enhancer = ImageEnhance.Brightness(img)
                bright_img = enhancer.enhance(1.5)  # Increase brightness by 50%
                bright_name = f"{base_name}_bright{ext}"
                bright_path = os.path.join(class_path, bright_name)
                bright_img.save(bright_path)
                
                # Random color enhancement
                color_enhancer = ImageEnhance.Color(img)
                color_img = color_enhancer.enhance(1.2)  # Slightly increase color
                color_name = f"{base_name}_color{ext}"
                color_path = os.path.join(class_path, color_name)
                color_img.save(color_path)
            
            except Exception as e:
                print(f"Error processing image {image_path}: {e}")

In [7]:
# Preprocess and augment Train, Validation, and Test datasets
print("\nProcessing and augmenting Train dataset...")
preprocess_and_augment(train_dir)

print("Processing and augmenting Validation dataset...")
preprocess_and_augment(validation_dir)

print("Processing and augmenting Test dataset...")
preprocess_and_augment(test_dir)

# Count and display images after augmentation
print("\nCounting images after preprocessing and augmentation...")
total_train_after = count_images(train_dir)
total_validation_after = count_images(validation_dir)
total_test_after = count_images(test_dir)

print(f"Train: {total_train_after} images")
print(f"Validation: {total_validation_after} images")
print(f"Test: {total_test_after} images")


Processing and augmenting Train dataset...
Processing and augmenting Validation dataset...
Processing and augmenting Test dataset...

Counting images after preprocessing and augmentation...
Train: 0 images
Validation: 0 images
Test: 0 images
